In [2]:
import gym
import numpy as np

In [3]:


env = gym.make('CartPole-v0')
for i_episode in range(1):
    observation = env.reset()
    for t in range(100):
        env.render()
        print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

[2018-01-08 00:17:06,558] Making new env: CartPole-v0


[-0.02454235 -0.01785042 -0.0492103  -0.03288961]
[-0.02489936 -0.2122334  -0.0498681   0.24386989]
[-0.02914403 -0.01643595 -0.0449907  -0.06411633]
[-0.02947274  0.17930122 -0.04627302 -0.37064777]
[-0.02588672 -0.01513382 -0.05368598 -0.09290682]
[-0.0261894   0.18071487 -0.05554412 -0.40203271]
[-0.0225751   0.37657885 -0.06358477 -0.71169716]
[-0.01504352  0.57252096 -0.07781871 -1.0236977 ]
[-0.0035931   0.76858826 -0.09829267 -1.33976422]
[ 0.01177866  0.96480069 -0.12508795 -1.66151377]
[ 0.03107468  1.16113829 -0.15831823 -1.99039914]
[ 0.05429744  1.35752648 -0.19812621 -2.32764867]
Episode finished after 12 timesteps


In [4]:
print(env.action_space)
print(env.observation_space)
print(env.observation_space.high)
print(env.observation_space.low)

Discrete(2)
Box(4,)
[  4.80000000e+00   3.40282347e+38   4.18879020e-01   3.40282347e+38]
[ -4.80000000e+00  -3.40282347e+38  -4.18879020e-01  -3.40282347e+38]


In [5]:
x = np.array([-2.4, 0, -4.0, 4, 2])
bins = np.linspace(-4.8, 4.8, 12)
out = np.digitize(x, bins)
print(bins)
print(out)

a = [np.linspace(env.observation_space.low[i], env.observation_space.high[i], 8) for i in range(4)]

print(len(a))
print(a)

[-4.8        -3.92727273 -3.05454545 -2.18181818 -1.30909091 -0.43636364
  0.43636364  1.30909091  2.18181818  3.05454545  3.92727273  4.8       ]
[ 3  6  1 11  8]
4
[array([-4.8       , -3.42857143, -2.05714286, -0.68571429,  0.68571429,
        2.05714286,  3.42857143,  4.8       ]), array([ -3.40282347e+38,  -2.43058819e+38,  -1.45835291e+38,
        -4.86117638e+37,   4.86117638e+37,   1.45835291e+38,
         2.43058819e+38,   3.40282347e+38]), array([-0.41887902, -0.2991993 , -0.17951958, -0.05983986,  0.05983986,
        0.17951958,  0.2991993 ,  0.41887902]), array([ -3.40282347e+38,  -2.43058819e+38,  -1.45835291e+38,
        -4.86117638e+37,   4.86117638e+37,   1.45835291e+38,
         2.43058819e+38,   3.40282347e+38])]


In [6]:
env = gym.make('CartPole-v0')
env.reset()


# Qlearner for CartPole
class QLearner():
    def __init__(self, env):
        self.env = env
        self.epsilon = 0.9
        self.alpha = 0.01
        self.gamma = 0.99
        
        # get initial state, divide continuous states into discrete bins
        self.bins = [np.linspace(env.observation_space.low[i], env.observation_space.high[i], 9) for i in range(4)]
        self.s = env.reset()
        self.s_bins = [np.digitize(self.s[i], self.bins[i]) for i in range(4)]
        
        #initialize Q
        self.Q = np.zeros((9, 9, 9, 9, 2))
#         print(self.bins)
        
    
    def getQ(self, s, action):
        return self.Q[s[0], s[1], s[2], s[3], action]
    
    
    def setQ(self, s, s_plus1, action, r):
        print("Setting Q: ")
        print("s: ", s, "s_plus1: ", s_plus1, " action: ", action, " r: ", r)
        q = self.getQ(s, action)
        max_q_splus1 = np.max(self.Q[s_plus1[0], s_plus1[1], s_plus1[2], s_plus1[3]])
        print("q: ", q, " max_q_splus1: ", max_q_splus1)
        new_q = q + self.alpha * (r + self.gamma * max_q_splus1 - q)
        
        print("new q: ", new_q)
        
        self.Q[s[0], s[1], s[2], s[3], action] = new_q
#         print(self.Q.shape)
#         print(self.Q[state[0], state[1], state[2], state[3]])
#         print(self.Q[3:6, 3:6, 3:6, 3:6, :])
    
    
    def step(self, epsilon=None):
        if not epsilon:
            epsilon = self.epsilon
        
        action = env.action_space.sample()
        s_plus1, r, done, _ = env.step(action)
        print("s0:", self.s)
        print("s1:", s_plus1)
        s_bins = [np.digitize(s_plus1[i], self.bins[i]) for i in range(4)]

        self.setQ(self.s_bins, s_bins, action, r)
        self.s_bins = s_bins
        
    
    def train(self):
        pass

    def getStats(self):
        pass


def test_qlearner():
    qlearner = QLearner(env)
    # print(env.reset())
    qlearner.step()



test_qlearner()



[2018-01-08 00:17:15,789] Making new env: CartPole-v0


s0: [-0.01862144  0.03762346 -0.0073345   0.01823664]
s1: [-0.01786897 -0.15739254 -0.00696977  0.30859646]
Setting Q: 
s:  [array(4), array(5), array(4), array(5)] s_plus1:  [array(4), array(4), array(4), array(5)]  action:  0  r:  1.0
q:  0.0  max_q_splus1:  0.0
new q:  0.01
